# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,souillac,-20.5167,57.5167,26.21,69,40,3.09,MU,1676227239
1,1,claveria,18.6074,121.0832,22.97,86,3,2.72,PH,1676227239
2,2,vaini,-21.2000,-175.2000,25.09,94,75,2.06,TO,1676227240
3,3,puerto leguizamo,-0.1934,-74.7819,32.93,46,55,1.62,CO,1676227240
4,4,barrow,71.2906,-156.7887,-26.99,76,0,6.69,US,1676227241


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng","Lat",geo= True, tiles= "OSM",
    frame_width = 700, frame_height = 500,
    size = "Humidity", scale = 0.8, color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21

#Wind speed less than 4.5 m/s

#Zero cloudiness

filtered_city_df = city_data_df

filtered_city_df = filtered_city_df.loc[(filtered_city_df["Max Temp"] < 27) & (filtered_city_df["Max Temp"] > 21)]
filtered_city_df = filtered_city_df[(filtered_city_df["Wind Speed"] < 4.5)]
filtered_city_df = filtered_city_df[(filtered_city_df["Cloudiness"] == 0)]


# Drop any rows with null values
filtered_city_df = filtered_city_df.dropna()

# Display sample data
filtered_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,140,yeppoon,-23.1333,150.7333,24.88,88,0,1.36,AU,1676227470
188,188,mecca,21.4267,39.8261,22.17,68,0,3.48,SA,1676227502
198,198,kavaratti,10.5669,72.6420,26.70,71,0,3.30,IN,1676227551
227,227,saint-pierre,-21.3393,55.4781,25.82,73,0,2.57,RE,1676227620
263,263,alice springs,-23.7000,133.8833,22.75,38,0,3.60,AU,1676227634
317,317,abu dhabi,24.4667,54.3667,22.06,60,0,1.54,AE,1676227624
330,330,salalah,17.0151,54.0924,24.05,53,0,1.03,OM,1676227780
343,343,dutse,11.7594,9.3392,24.40,14,0,4.40,NG,1676227784
508,508,blackwater,-23.5833,148.8833,24.74,60,0,2.91,AU,1676228080
523,523,malkapur,20.8833,76.2000,21.24,22,0,2.26,IN,1676228085


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = filtered_city_df.loc[:, ('City', 'Country','Lat','Lng','Humidity')]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
140,yeppoon,AU,-23.1333,150.7333,88,
188,mecca,SA,21.4267,39.8261,68,
198,kavaratti,IN,10.5669,72.6420,71,
227,saint-pierre,RE,-21.3393,55.4781,73,
263,alice springs,AU,-23.7000,133.8833,38,
317,abu dhabi,AE,24.4667,54.3667,60,
330,salalah,OM,17.0151,54.0924,53,
343,dutse,NG,11.7594,9.3392,14,
508,blackwater,AU,-23.5833,148.8833,60,
523,malkapur,IN,20.8833,76.2000,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {
    "categories" : categories,
    "radius" : radius,
    "apiKey" : geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yeppoon - nearest hotel: Blossom's on Seaspray
mecca - nearest hotel: Rehab Al Marwa Hotel 1
kavaratti - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
alice springs - nearest hotel: Aurora Alice Springs
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
salalah - nearest hotel: Muscat International Hotel
dutse - nearest hotel: No hotel found
blackwater - nearest hotel: No hotel found
malkapur - nearest hotel: No hotel found
lalmohan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
140,yeppoon,AU,-23.1333,150.7333,88,Blossom's on Seaspray
188,mecca,SA,21.4267,39.8261,68,Rehab Al Marwa Hotel 1
198,kavaratti,IN,10.5669,72.6420,71,No hotel found
227,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
263,alice springs,AU,-23.7000,133.8833,38,Aurora Alice Springs
317,abu dhabi,AE,24.4667,54.3667,60,Marriott Downtown Abu-Dhabi
330,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
343,dutse,NG,11.7594,9.3392,14,No hotel found
508,blackwater,AU,-23.5833,148.8833,60,No hotel found
523,malkapur,IN,20.8833,76.2000,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng","Lat",geo= True, tiles= "OSM",
    frame_width = 700, frame_height = 500,
    size = "Humidity", scale = 0.8, color = "City", hover_cols = ["City", "Country", "Hotel Name"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)